<a href="https://colab.research.google.com/github/wandern-bot/vanguard-fund-analyzer/blob/main/vanguard_fund_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vanguard Fund Data Scraping Project

This notebook provides a clean and organized version of the code used to scrape ETF and Mutual Fund data from the Vanguard website. It is designed to be easy to understand and use by others.
The output CSV is used as a data source for this interactive Tableau dashboard:
🎯 [Which Vanguard Funds Offer the Best Value?](https://public.tableau.com/app/profile/nian.liu6717/viz/Vanguard_Funds_Best_Bang_Buck_Interactive_Analysis/VanguardUniverse)

## Project Goal

The goal of this project is to extract detailed information about Vanguard ETFs and Mutual Funds, including performance metrics, expense ratios, and other key data points, by scraping the fund lists available on the Vanguard investor website.

## Libraries Used

I use the following libraries:

*   **Selenium:** For automating browser interaction to load dynamic content and navigate through pagination.
*   **BeautifulSoup:** For parsing the HTML content obtained from Selenium to extract data.
*   **Pandas:** For structuring the extracted data into DataFrames and performing data cleaning and manipulation.
*   **Webdriver Manager:** To simplify the process of setting up and managing the Chrome WebDriver.

## Setup: Google Drive Mount and Library Installations

This section includes mounting Google Drive to save the output CSV file and installing the necessary libraries for web scraping.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Installation of Libraries and Chrome
# These libraries and Chrome are necessary for web scraping the Vanguard website using Selenium.
# `requests`, `pandas`, `beautifulsoup4`, `lxml` are for data handling and parsing.
# `selenium` is the browser automation tool.
# `webdriver-manager` automatically manages the Chrome WebDriver executable.
# We also install Google Chrome Stable as the browser for Selenium to control.
!pip install requests pandas beautifulsoup4 lxml selenium webdriver-manager

# Install a compatible version of Chrome for Selenium to use
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -y -f # Install dependencies

(Reading database ... 126464 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (138.0.7204.92-1) over (138.0.7204.92-1) ...
Setting up google-chrome-stable (138.0.7204.92-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


## Scraping Function

The `scrape_vanguard_funds` function utilizes Selenium to navigate the given URL, handle dynamic content and pagination, and extract data from the table. It also includes data cleaning steps to format the extracted information into a structured Pandas DataFrame.

**Function Breakdown:**

*   **WebDriver Setup:** Initializes the Chrome WebDriver in headless mode.
*   **Pagination Loop:** Iterates through the pages of the fund list.
*   **Explicit Waits:** Waits for key elements (like fund names and the table body) to ensure the page content is loaded before scraping.
*   **Data Extraction:** Uses BeautifulSoup to parse the page source and extract data points based on predefined CSS selectors.
*   **Pagination Navigation:** Identifies and clicks the "Next" pagination button. Includes error handling and retry mechanisms for clicking the button.
*   **Data Cleaning:** Cleans and formats the extracted data, converting percentages and currency strings to numeric types, handling missing values, and renaming columns.
*   **Returns DataFrame:** Returns a concatenated DataFrame containing data from all pages.

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def scrape_vanguard_funds(url, fund_type):
    """
    Scrapes fund data from a given Vanguard URL using Selenium.

    Args:
        url (str): The URL of the Vanguard fund list page.
        fund_type (str): The type of fund being scraped (e.g., "ETF", "Mutual Fund").

    Returns:
        pandas.DataFrame: A DataFrame containing the scraped and cleaned fund data.
    """
    # Set up Selenium WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode
    options.add_argument('--no-sandbox') # Bypass OS security model
    options.add_argument('--disable-dev-shm-usage') # Overcome limited resource problems
    # Specify the path to the installed Chrome binary (common in Colab)
    options.binary_location = '/usr/bin/google-chrome'


    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)

    all_fund_data = []
    page_num = 1

    # Define the data points to extract and their selectors based on data-rpa-tag-id
    # These selectors were identified by inspecting the webpage's HTML.
    data_points_to_extract = {
        "Symbol": 'td [data-rpa-tag-id="symbol"], p[data-rpa-tag-id="symbol"], span[data-rpa-tag-id="symbol"], a[data-rpa-tag-id="symbol"]',
        "Fund Name": 'td [data-rpa-tag-id="longName"], p[data-rpa-tag-id="longName"], span[data-rpa-tag-id="longName"], a[data-rpa-tag-id="longName"]',
        "Asset Class": 'td [data-rpa-tag-id="assetClass"], p[data-rpa-tag-id="assetClass"], span[data-rpa-tag-id="assetClass"], a[data-rpa-tag-id="assetClass"]',
        "Risk": 'td [data-rpa-tag-id="risk"], p[data-rpa-tag-id="risk"], span[data-rpa-tag-id="risk"], a[data-rpa-tag-id="risk"]',
        "Expense Ratio": 'td [data-rpa-tag-id="expenseRatio"], p[data-rpa-tag-id="expenseRatio"], span[data-rpa-tag-id="expenseRatio"], a[data-rpa-tag-id="expenseRatio"]',
        "Yield Pct": 'td [data-rpa-tag-id="yieldPct"], p[data-rpa-tag-id="yieldPct"], span[data-rpa-tag-id="yieldPct"], a[data-rpa-tag-id="yieldPct"]',
        "YTD Percent": 'td [data-rpa-tag-id="ytdPercent"], p[data-rpa-tag-id="ytdPercent"], span[data-rpa-tag-id="ytdPercent"], a[data-rpa-tag-id="ytdPercent"]',
        "One Yr Pct": 'td [data-rpa-tag-id="oneYrPct"], p[data-rpa-tag-id="oneYrPct"], span[data-rpa-tag-id="oneYrPct"], a[data-rpa-tag-id="oneYrPct"]',
        "Five Yr Pct": 'td [data-rpa-tag-id="fiveYrPct"], p[data-rpa-tag-id="fiveYrPct"], span[data-rpa-tag-id="fiveYrPct"], a[data-rpa-tag-id="fiveYrPct"]',
        "Ten Yr Pct": 'td [data-rpa-tag-id="tenYrPct"], p[data-rpa-tag-id="tenYrPct"], span[data-rpa-tag-id="tenYrPct"], a[data-rpa-tag-id="tenYrPct"]',
        "Inception Percentage": 'td [data-rpa-tag-id="inceptionPercentage"], p[data-rpa-tag-id="inceptionPercentage"], span[data-rpa-tag-id="inceptionPercentage"], a[data-rpa-tag-id="inceptionPercentage"]',
        "Inception Date": 'td [data-rpa-tag-id="inceptionDate"], p[data-rpa-tag-id="inceptionDate"], span[data-rpa-tag-id="inceptionDate"], a[data-rpa-tag-id="inceptionDate"]',
        "Min Investment": 'td [data-rpa-tag-id="minInvestment"], p[data-rpa-tag-id="minInvestment"], span[data-rpa-tag-id="minInvestment"], a[data-rpa-tag-id="minInvestment"]',
        "Price NAV": 'td [data-rpa-tag-id="priceNAV"], p[data-rpa-tag-id="priceNAV"], span[data-rpa-tag-id="priceNAV"], a[data-rpa-tag-id="priceNAV"]',
        "Price Change NAV": 'td [data-rpa-tag-id="priceChangeNAV"], p[data-rpa-tag-id="priceChangeNAV"], span[data-rpa-tag-id="priceChangeNAV"], a[data-rpa-tag-id="priceChangeNAV"]',
        "Price Percent NAV": 'td [data-rpa-tag-id="pricePercentNAV"], p[data-rpa-tag-id="pricePercentNAV"], span[data-rpa-tag-id="pricePercentNAV"], a[data-rpa-tag-id="pricePercentNAV"]'
    }

    # --- Main Scraping Loop with Pagination ---
    while True:
        print(f"Scraping page {page_num}...")
        try:
            # Add specific explicit wait for a key data element (like Fund Name) to ensure content is loaded
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'td a[data-rpa-tag-id="longName"], p a[data-rpa-tag-id="longName"], span a[data-rpa-tag-id="longName"]'))
            )
            print("Key data element (Fund Name) found on the webpage.")

            # Add explicit wait for the tbody to be present within the table
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'table tbody'))
            )
            print("tbody found within the table.")


            # Use BeautifulSoup with the page source after waiting for elements
            soup = BeautifulSoup(driver.page_source, "lxml")

            table = soup.find("table")

            if table is None:
                print(f"Error: Table is None after getting page source on page {page_num}. Exiting pagination.")
                break # Exit loop if table is not found


            # Find the table body (tbody) which typically contains data rows
            tbody = table.find("tbody")

            if tbody is None:
                print(f"Error: Could not find tbody in the table on page {page_num}. Exiting pagination.")
                break # Exit loop if tbody is not found


            rows = tbody.find_all("tr")

            print(f"Found {len(rows)} rows in tbody on page {page_num}.")


            current_page_data = []
            for row in rows:
                row_data = {}
                # Attempt to extract data for each defined data point using its selector
                for data_point, selector in data_points_to_extract.items():
                    # Find the element within the current row using the specific selector
                    element = row.select_one(selector)
                    if element:
                        row_data[data_point] = element.text.strip()
                    else:
                        row_data[data_point] = None # Set to None if the element is not found

                # Append data from the current page to the overall list
                current_page_data.append(row_data)

            all_fund_data.extend(current_page_data)
            print(f"Total rows collected so far: {len(all_fund_data)}")


            # --- Pagination Logic ---
            # Identify and click the "Next" button to navigate to the next page
            next_element = None
            try:
                # Use the identified selector for the "Next" button
                next_element = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="next page"]'))
                )
            except TimeoutException:
                print("Could not find a clickable Next pagination element within timeout.")


            # Check if the next element is present and not disabled (indicating end of pagination)
            try:
                next_button_check = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="next page"]')
                if "disabled" in next_button_check.get_attribute("class"):
                    print("Next element is disabled. End of pagination.")
                    break # Exit loop if the next button is disabled
            except NoSuchElementException:
                print("Next button element not found during disabled check. End of pagination.")
                break # Exit loop if the next button is not found


            if next_element:
                # Click the "Next" element
                # Use JavaScript click as a fallback if direct click fails due to interception
                try:
                    next_element.click()
                    print("Clicked next pagination element.")
                except ElementClickInterceptedException:
                    print("Element click intercepted, attempting JavaScript click.")
                    driver.execute_script("arguments[0].click();", next_element)
                    print("Clicked next pagination element using JavaScript.")

                page_num += 1
                # Add a short sleep to allow page transition and loading
                time.sleep(3) # Increased sleep time to allow for page load before next iteration
            else:
                print("No clickable Next pagination element found (after disabled check). End of pagination.")
                break # Exit loop if next element is not found (should be covered by disabled check/NoSuchElementException)


        except TimeoutException:
            print("Timeout waiting for key data element or tbody on the page. Exiting loop.")
            break
        except NoSuchElementException:
            print("A required element (table, tbody, or next button) was not found. Exiting loop.")
            break
        except ElementNotInteractableException:
            print("Next element not interactable. End of pagination.")
            break
        except Exception as e:
            print(f"An unexpected error occurred during scraping page {page_num}: {e}")
            break


    driver.quit() # Close the browser when scraping is complete

    # --- Data Cleaning and Formatting ---
    # Process the collected data into a Pandas DataFrame and apply cleaning steps
    if all_fund_data:
        df = pd.DataFrame(all_fund_data)

        # Clean column names (remove newlines and leading/trailing spaces)
        df.columns = [col.replace("\n", " ").strip() for col in df.columns]

        # Define columns that should be converted to numeric and divided by 100 (these are percentages)
        percent_cols_divide_100 = [
            "Yield Pct", "YTD Percent", "One Yr Pct", "Five Yr Pct",
            "Ten Yr Pct", "Inception Percentage", "Price Percent NAV", "Expense Ratio"
            ]

        # Define other numeric columns (not divided by 100)
        other_numeric_cols = [
             "Min Investment", "Price NAV", "Price Change NAV"
            ]

        all_numeric_cols = percent_cols_divide_100 + other_numeric_cols

        for col in all_numeric_cols:
            try:
                if col in df.columns and df[col].dtype == 'object':
                     # Specific handling for "Price Percent NAV" to remove parentheses
                     if col == "Price Percent NAV":
                         df[col] = df[col].str.replace('(', '', regex=False).str.replace(')', '', regex=False)

                     # Replace '—' with '0' before attempting conversion
                     df[col] = df[col].str.replace('—', '0').str.replace(',', '')

                     # Specific handling for "Min Investment" to process K/M
                     if col == "Min Investment":
                         # Remove '$' before processing K/M
                         value_str = df[col].str.replace('$', '', regex=False)
                         # Replace K with *1000 and M with *1000000, then evaluate the string
                         value_str = value_str.str.replace('K', '*1000', regex=False).str.replace('M', '*1000000', regex=False)
                         # Handle potential errors during eval (e.g., ranges like "1M - 5M")
                         df[col] = value_str.apply(lambda x: eval(x) if isinstance(x, str) and ('*' in x or '.' in x or x.replace('.', '', 1).isdigit()) else None) # Improved check for numeric like strings


                     else: # General numeric cleaning for other columns
                          # Remove '%' and '$' for conversion, then convert to float
                          # Ensure these replacements are applied only if the symbols are present
                          df[col] = df[col].str.replace('%', '', regex=False).str.replace('$', '', regex=False).astype(float)


                     # Divide by 100 only for specific percentage columns
                     if col in percent_cols_divide_100:
                         df[col] = df[col] / 100

            except Exception as e:
                print(f"Could not process column {col} for numeric conversion: {e}")
                # If conversion fails, set to None or keep original value (depends on desired outcome)
                if col in df.columns:
                    df[col] = None

        # Apply rounding to all numeric columns after conversion
        for col in df.select_dtypes(include=['float64', 'int64']).columns:
            df[col] = df[col].round(4)


        # Specific cleaning for Inception Date to remove parentheses
        if "Inception Date" in df.columns:
            try:
                if df["Inception Date"].dtype == 'object':
                    df["Inception Date"] = df["Inception Date"].str.replace('(', '', regex=False).str.replace(')', '', regex=False).str.strip()
                    # Derive Inception Year
                    df['Inception Year'] = pd.to_datetime(df['Inception Date'], errors='coerce').dt.year
            except Exception as e:
                print(f"Could not process Inception Date column: {e}")
                # Handle error, e.g., keep original or set to None


        # Rename columns to be more descriptive
        if "Yield Pct" in df.columns:
            df.rename(columns={"Yield Pct": "SEC Yield"}, inplace=True)
        if "Price Change NAV" in df.columns:
            df.rename(columns={"Price Change NAV": "Daily NAV Chg"}, inplace=True)
        if "Price Percent NAV" in df.columns:
            df.rename(columns={"Price Percent NAV": "Daily NAV Pct Chg"}, inplace=True)

        df["Fund Type"] = fund_type # Assign the fund type parameter

        # Set display format for float columns to 4 decimal places
        # This affects how the DataFrame is displayed, not the underlying data
        # Apply this after potential conversion errors might have introduced None
        for col in df.select_dtypes(include=['float64']).columns:
             # Apply format to all float columns, including those not in numeric_cols if any were created
             pd.options.display.float_format = '{:,.4f}'.format


        return df
    else:
        return pd.DataFrame() # Return empty DataFrame if no data was collected

## Executing the Scraper and Saving Data

This section defines the URLs for the ETF and Mutual Fund lists, calls the `scrape_vanguard_funds` function for each, saves the resulting DataFrames to CSV files, and displays the head of each DataFrame along with the number of rows extracted.

In [ ]:
# Define URLs for ETF and Mutual Fund lists
etf_url = "https://investor.vanguard.com/investment-products/list/etfs?filters=open"
mutual_fund_url = "https://investor.vanguard.com/investment-products/list/mutual-funds?filters=open"

# Scrape and clean ETF data
print("--- Scraping ETF Data ---")
df_all_etfs = scrape_vanguard_funds(etf_url, "ETF")
df_all_etfs.to_csv("vanguard_etfs.csv", index=False)
print("\n--- ETF Data Preview ---")
display(df_all_etfs.head())
print(f"Number of extracted ETF rows: {len(df_all_etfs)}")

# Scrape and clean Mutual Fund data
print("\n--- Scraping Mutual Fund Data ---")
df_mutual_funds = scrape_vanguard_funds(mutual_fund_url, "Mutual Fund")
df_mutual_funds.to_csv("vanguard_mutual_funds.csv", index=False)
print("\n--- Mutual Fund Data Preview ---")
display(df_mutual_funds.head())
print(f"Number of extracted Mutual Fund rows: {len(df_mutual_funds)}")

--- Scraping ETF Data ---
Scraping page 1...
Key data element (Fund Name) found on the webpage.
tbody found within the table.
Found 50 rows in tbody on page 1.
Total rows collected so far: 50
Element click intercepted, attempting JavaScript click.
Clicked next pagination element using JavaScript.
Scraping page 2...
Key data element (Fund Name) found on the webpage.
tbody found within the table.
Found 44 rows in tbody on page 2.
Total rows collected so far: 94
Could not find a clickable Next pagination element within timeout.
No clickable Next pagination element found (after disabled check). End of pagination.

--- ETF Data Preview ---


,Symbol,Fund Name,Asset Class,Risk,Expense Ratio,SEC Yield,YTD Percent,One Yr Pct,Five Yr Pct,Ten Yr Pct,Inception Percentage,Inception Date,Min Investment,Price NAV,Daily NAV Chg,Daily NAV Pct Chg,Inception Year,Fund Type
0,VBIL,0-3 Month Treasury Bill ETF,Bond - Short-term Investment,1,0.0007,0.0421,0.0000,0.0000,0.0000,0.0000,0.0163,02/07/2025,1,75.3600,0.0300,0.0004,2025,ETF
1,VTEC,California Tax-Exempt Bond ETF,Bond - Inter-term State Muni,2,0.0008,0.0362,-0.0042,0.0108,0.0000,0.0000,0.0099,01/26/2024,1,97.1400,0.0100,0.0001,2024,ETF
2,VOX,Communication Services ETF,Stock - Sector,5,0.0009,0.0089,0.1077,0.2522,0.1404,0.0905,0.0865,09/23/2004,1,171.0200,1.0800,0.0064,2004,ETF
3,VCR,Consumer Discretionary ETF,Stock - Sector,5,0.0009,0.0070,-0.0115,0.1698,0.1387,0.1272,0.1096,01/26/2004,1,369.5200,2.2000,0.0060,2004,ETF
4,VDC,Consumer Staples ETF,Stock - Sector,5,0.0009,0.0220,0.0624,0.1033,0.1060,0.0860,0.0954,01/26/2004,1,222.1600,0.3200,0.0014,2004,ETF


Number of extracted ETF rows: 94

--- Scraping Mutual Fund Data ---
Scraping page 1...
Key data element (Fund Name) found on the webpage.
tbody found within the table.
Found 50 rows in tbody on page 1.
Total rows collected so far: 50
Element click intercepted, attempting JavaScript click.
Clicked next pagination element using JavaScript.
Scraping page 2...
Key data element (Fund Name) found on the webpage.
tbody found within the table.
Found 50 rows in tbody on page 2.
Total rows collected so far: 100
Element click intercepted, attempting JavaScript click.
Clicked next pagination element using JavaScript.
Scraping page 3...
Key data element (Fund Name) found on the webpage.
tbody found within the table.
Found 50 rows in tbody on page 3.
Total rows collected so far: 150
Element click intercepted, attempting JavaScript click.
Clicked next pagination element using JavaScript.
Scraping page 4...
Key data element (Fund Name) found on the webpage.
tbody found within the table.
Found 50 rows 

,Symbol,Fund Name,Asset Class,Risk,Expense Ratio,SEC Yield,YTD Percent,One Yr Pct,Five Yr Pct,Ten Yr Pct,Inception Percentage,Inception Date,Min Investment,Price NAV,Daily NAV Chg,Daily NAV Pct Chg,Inception Year,Fund Type
0,VFIAX,500 Index Fund Admiral Shares,Stock - Large-Cap Blend,4,0.0004,0.0118,0.0748,0.1512,0.1660,0.1360,0.0838,11/13/2000,"3,000.0000",579.5700,4.8800,0.0085,2000,Mutual Fund
1,VADGX,Advice Select Dividend Growth Fund,Stock - Large-Cap Blend,5,0.0041,0.0098,0.0495,0.0984,0.0000,0.0000,0.0665,11/09/2021,0.0000,30.4300,0.2100,0.0069,2021,Mutual Fund
2,VAGVX,Advice Select Global Value Fund,International,5,0.0040,0.0192,0.1160,0.1511,0.0000,0.0000,0.0675,11/09/2021,0.0000,29.0600,0.1300,0.0045,2021,Mutual Fund
3,VAIGX,Advice Select International Growth Fund,International,5,0.0040,0.0004,0.2084,0.2874,0.0000,0.0000,-0.0155,11/09/2021,0.0000,23.2500,0.1000,0.0043,2021,Mutual Fund
4,VBPIX,Baillie Gifford Global Positive Impact Stock F...,International,4,0.0059,0.0026,0.1452,0.1794,0.0869,0.0000,0.1331,12/14/2017,"3,000.0000",21.9600,0.0900,0.0041,2017,Mutual Fund


Number of extracted Mutual Fund rows: 267
